In [11]:
'''
 Yacine Mahdid 2019-11-02
 This script was created for the analysis of the graph features for consciousness decoding
 The goal here is to take the CSV created in the preprocess.m file and to manipulate it to get labels, participant id and such.
 This script was tested with atruncated version of the full dataset (only alpha, only ec1 and only 1 null network) while the real dataset was getting generated

'''

'\n Yacine Mahdid 2019-11-02\n This script was created for the analysis of the graph features for consciousness decoding\n The goal here is to take the CSV created in the preprocess.m file and to manipulate it to get labels, participant id and such.\n This script was tested with atruncated version of the full dataset (only alpha, only ec1 and only 1 null network) while the real dataset was getting generated\n\n'

In [7]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

# First step will be to import the feature data and the labels
data_path = "C:/Users/biapt/Documents/only_bin_data.csv";
data = pd.read_csv(data_path)
# here what everything means:
'''
frequencies = {"alpha","beta","delta","theta","gamma"};
epochs = {'ec1','emf5','ec1','if5','emf5','eml5','ec3','ec8'};
graph = {"aec","pli"}
The ids we see are mapping to these
'''
data

,p_id,frequency,epoch,graph,window,clust_coeff_ 1,clust_coeff_ 2,clust_coeff_ 3,clust_coeff_ 4,clust_coeff_ 5,...,clust_coeff_ 77,clust_coeff_ 78,clust_coeff_ 79,clust_coeff_ 80,clust_coeff_ 81,clust_coeff_ 82,norm_avg_clust_coeff,norm_g_eff,community,small_worldness
0,1,1,1,0,1,0.248370,0.280700,0.333330,0.000000,0.166670,...,0.000000,0.000000,0.000000,0.000000,0.333330,0.000000,0.76157,0.92896,0.28682,0.65412
1,1,1,1,1,1,0.190480,0.400000,0.000000,0.083333,0.105260,...,0.333330,0.025641,0.100000,0.027778,0.000000,0.066667,1.13490,0.95472,0.35036,1.05650
2,1,1,1,0,2,0.285710,0.300000,0.222220,0.222220,0.533330,...,0.325000,0.181820,0.000000,0.581820,0.000000,0.000000,1.07940,0.98457,0.26266,1.05200
3,1,1,1,1,2,0.044444,0.266670,0.100000,0.090909,0.077922,...,0.400000,0.171430,0.109890,0.066667,0.000000,0.216370,0.75411,0.96120,0.33885,0.70654
4,1,1,1,0,3,0.000000,0.102940,0.054945,0.155560,0.000000,...,0.200000,0.071429,0.000000,0.219050,0.305560,0.153850,1.00300,0.96710,0.38174,0.95572
5,1,1,1,1,3,0.200000,0.000000,0.166670,0.089744,0.219050,...,0.314290,0.212120,0.055556,0.100000,0.000000,0.409090,0.93900,0.95890,0.28893,0.88198
6,1,1,1,0,4,0.000000,0.116670,0.100000,0.190480,0.000000,...,0.166670,0.200000,0.000000,0.012821,0.600000,0.151520,0.69130,0.92776,0.28797,0.60642
7,1,1,1,1,4,0.333330,0.000000,0.000000,0.000000,0.422220,...,0.166670,0.199130,0.000000,0.000000,0.333330,0.254550,0.88075,0.96139,0.29421,0.82491
8,1,1,1,0,5,0.190480,0.071429,0.166670,0.261900,0.000000,...,0.000000,0.000000,0.400000,0.333330,0.000000,0.000000,0.77177,0.93747,0.25418,0.68024
9,1,1,1,1,5,0.100000,0.000000,0.000000,0.250000,0.205880,...,0.000000,0.172310,0.000000,0.357140,0.333330,0.200000,0.97411,0.96169,0.36066,0.92028


In [9]:
# We will keep the same structure than we did in the AEC vs wPLI project for the machine learning

# Machine Learning 
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# Sklearn utils
from sklearn.base import clone
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix

# aec = 0 pli = 1
analysis_type = 1
epochs_to_keep = [1,3]
analysis = data['graph'] == analysis_type
baseline = data['epoch'] == epochs_to_keep[0]
other = data['epoch'] == epochs_to_keep[1]
subset_data = data[analysis & (baseline | other)]

clfs = [LinearDiscriminantAnalysis(solver='svd'), SVC(kernel='linear', C=0.1),SVC(kernel='linear', C=0.5), SVC(kernel='linear', C=1.0), SVC(kernel='rbf', C=0.1), SVC(kernel='rbf',C=1.0)]
selected_clf = clfs[3]

def classify(dataset, original_clf):
    clf = clone(original_clf)
    # Initialize the Result data structures
    cms = []
    accuracies = []
    reports = []
    # TODO: Check in the MATLAB file how to not have to do this + 1
    for test_id in range(1,10):
        print("Participant: " + str(test_id) + " in hold-out set:")
        
        # Split the data in a leave one subject out manner
        
        # Get the training and test dataset
        training_dataset = dataset[dataset['p_id'] != test_id]
        test_dataset = dataset[dataset['p_id'] == test_id]
        
        # Get the X and Y 
        X_train = training_dataset.iloc[:,5:]
        y_train = training_dataset['epoch']
        
        # Training the min max normalizer
        min_max_scaler = MinMaxScaler()
        min_max_scaler.fit(X_train)
        X_train = min_max_scaler.transform(X_train)
        
        X_test = test_dataset.iloc[:,5:]
        y_test = test_dataset['epoch']
        
        # Normalize the test set using a training scaler
        X_test = min_max_scaler.transform(X_test)
        
        # Fitting our model
        clf.fit(X_train, y_train)

        # predicting
        y_pred = clf.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)

        report = classification_report(y_test, y_pred, output_dict=True)
        accuracy = accuracy_score(y_test, y_pred)
        print("Generalization accuracy: " + str(accuracy))
        print(cm)
        print(report)
        
        # Saving the results
        cms.append(cm)
        accuracies.append(accuracy)
        reports.append(report)
        
    return (cms,accuracies,reports)

(cms, accuracies, report) = classify(subset_data, selected_clf)
print("Mean accuracy is: " + str(np.mean(accuracies)))

Participant: 1 in hold-out set:
Generalization accuracy: 0.4608695652173913
[[23 34]
 [28 30]]
{'1': {'precision': 0.45098039215686275, 'recall': 0.40350877192982454, 'f1-score': 0.42592592592592593, 'support': 57}, '3': {'precision': 0.46875, 'recall': 0.5172413793103449, 'f1-score': 0.4918032786885246, 'support': 58}, 'accuracy': 0.4608695652173913, 'macro avg': {'precision': 0.45986519607843135, 'recall': 0.46037507562008473, 'f1-score': 0.45886460230722526, 'support': 115}, 'weighted avg': {'precision': 0.45994245524296673, 'recall': 0.4608695652173913, 'f1-score': 0.4591510255801061, 'support': 115}}
Participant: 2 in hold-out set:
Generalization accuracy: 0.5826086956521739
[[29 28]
 [20 38]]
{'1': {'precision': 0.5918367346938775, 'recall': 0.5087719298245614, 'f1-score': 0.5471698113207547, 'support': 57}, '3': {'precision': 0.5757575757575758, 'recall': 0.6551724137931034, 'f1-score': 0.6129032258064515, 'support': 58}, 'accuracy': 0.5826086956521739, 'macro avg': {'precision'

## Result
- We get something that is close to 50% accuracy for some participant while being a bit above for others
- This might be due by the clustering coefficient per regions behing not too important (only the global help)
- We will augment this dataset with the standard deviations and the means from the previous study to see if that helps
- The fact that the graph was binarized might have lowered the classification (we will see if that is the case with the mean and standard deviation).

## Result after 2nd Analysis
### Now (with clust coeff per region)
#### Unconscious
- aec = 0.8710220804490129
- pli = 0.7523399801868711

#### pre-ROC
- aec = 0.7134612273470387 
- pli = 0.7783826838738702

### Before:
#### Unconscious
- aec = 0.8689650608531154
- pli = 0.7868548852868595

#### Pre-ROC
- aec = 0.7274831932263043
- pli = 0.8041592563476617

### Reduced (no clust coeff per region):
#### Unconscious
- aec = 0.8751687095710673
- pli = 0.7772391411037457

#### pre-ROC
- aec = 0.7277764758615426
- pli = 0.8092161019771724

## Bottomline
Bottom line seems to be that the mean and standard deviation are driving the classification. The new features are helping but only marginally. The clustering coefficient per region is not helping much and isn't a replacement for mean and standard deviation. If we remove the mean the wPLI gets a hit in classification accuracy while the AEC get a boost of almost 2 points. This make sense if we look at the figure though of the feature importance. However, we did this whole analysis for the graph theory using a binarized graph, the graph is not binarized for the mean and standard deviation. We also know that the small weights are contains important information as shown by the standard deviation feature which is driving most of the classification accuracy and with the motif analysis.
## Next Steps
- Find weighted version of your graph theory features and use them for classification
- Add in analysis of the theta band
- Add in more graph theory features
- There is also very hihg fluctuation between participant, there might a feature that could help us reduce this fluctuation.